# Automated Gleason Grading

## Imports

In [1]:
import os

import openslide

# Torch packages
import torch
from torch.utils.data import Dataset
from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms

from focal_loss import FocalLoss
from torch.nn import BCEWithLogitsLoss

# General packages
import pandas as pd
import numpy as np
import numpy.random as random
import matplotlib
import matplotlib.pyplot as plt
import math
import time
from tqdm import tqdm
import pickle

## Initialize Globals and Load Datasets

In [2]:
# define the number of channels in the input, number of classes,
# and number of levels in the U-Net model
NUM_CHANNELS = 1
NUM_CLASSES = 6
NUM_LEVELS = 3
# initialize learning rate, number of epochs to train for, and the batch size
INIT_LR = 0.001
NUM_EPOCHS = 40
BATCH_SIZE = 32
NUM_PSEUDO_EPOCHS = 1024
# define the input image dimensions
PATCH_WIDTH = 256 #256
PATCH_HEIGHT = 256
# define threshold to filter weak predictions
THRESHOLD = 0.5

# define the validation split
VAL_SPLIT = 0.85

# define the test split
TEST_SPLIT = 0.95

# define the path to the base output directory
BASE_OUTPUT = "./output"
# define the path to the output serialized model, model training
# plot, and testing image paths
MODEL_PATH = f"{BASE_OUTPUT}/unet_tgs_salt.pth"
PLOT_PATH = f"{BASE_OUTPUT}/plot.png"
TEST_PATHS = f"{BASE_OUTPUT}/test_paths.txt"

# Location of the training images
DATA_PATH = '../../ganz/data/panda_dataset'

# image and mask directories
data_dir = f'{DATA_PATH}/train_images'
mask_dir = f'{DATA_PATH}/train_label_masks'

# Location of training labels
train = pd.read_csv(f'{DATA_PATH}/train.csv').set_index('image_id')
test = pd.read_csv(f'{DATA_PATH}/test.csv').set_index('image_id')
submission = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')

# # Create training image name list
# all_train_img_names = list(train.index)[:12]

# Get mask thumbnail dictionary
thumbnail_filename = "./data/thumbnails_" + str(PATCH_WIDTH) + "x" + str(PATCH_HEIGHT) + ".p"
with open(thumbnail_filename, "rb") as fp:
    thumbnails_dict = pickle.load(fp)

# determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print(DEVICE)
# # determine if we will be pinning memory during data loading
# PIN_MEMORY = True if DEVICE == "cuda" else False

# color map for printing masks
cmap = matplotlib.colors.ListedColormap(['black', 'gray', 'green', 'yellow', 'orange', 'red'])

# Sample images
# all_train_img_names = [
#     '07a7ef0ba3bb0d6564a73f4f3e1c2293',
#     '037504061b9fba71ef6e24c48c6df44d',
#     '035b1edd3d1aeeffc77ce5d248a01a53',
#     '059cbf902c5e42972587c8d17d49efed',
#     '06a0cbd8fd6320ef1aa6f19342af2e68',
#     '06eda4a6faca84e84a781fee2d5f47e1',
#     '0a4b7a7499ed55c71033cefb0765e93d',
#     '0838c82917cd9af681df249264d2769c',
#     '046b35ae95374bfb48cdca8d7c83233f',
#     '074c3e01525681a275a42282cd21cbde',
#     '05abe25c883d508ecc15b6e857e59f32',
#     '05f4e9415af9fdabc19109c980daf5ad',
#     '060121a06476ef401d8a21d6567dee6d',
#     '068b0e3be4c35ea983f77accf8351cc8',
#     '08f055372c7b8a7e1df97c6586542ac8'
# ]

## Patch Segmentation

### Dataset Class

In [3]:
class SegmentationDataset(Dataset):
  def __init__(self, 
              wsi_names, 
              mask_thumbnails,
              pseudo_epoch_length: int = 1024, 
              transformations = None):
    self.wsi_names = wsi_names
    self.mask_thumbnails = mask_thumbnails
    self.pseudo_epoch_length = pseudo_epoch_length

    # opens all slides and stores them in slide_dict
    self.slide_dict = self.make_slide_dict(wsi_names=self.wsi_names)

    # samples a list of patch coordinates and annotations 
    self.sample_dict = self.sample_coord_list(pseudo_epoch_length=self.pseudo_epoch_length)

    if transformations is not None:
      self.transformations = transformations
    else:
      self.transformations = transforms.Compose([transforms.ToTensor()])

  def make_slide_dict(self, wsi_names):
    slide_dict = {}
    bad_samples = []
    for wsi_name in tqdm(wsi_names, total=len(wsi_names), desc='Make Slide Dict'):
      if wsi_name not in slide_dict:
        slide_path = os.path.join(data_dir, f'{wsi_name}.tiff')
        mask_path = os.path.join(mask_dir, f'{wsi_name}_mask.tiff')
        if os.path.exists(slide_path) and os.path.exists(mask_path):
          slide_dict[wsi_name] = {}
          slide_dict[wsi_name]['slide'] = openslide.OpenSlide(slide_path)
          slide_dict[wsi_name]['mask'] = openslide.OpenSlide(mask_path)
          slide_dict[wsi_name]['size'] = slide_dict[wsi_name]['slide'].dimensions
        else:
          bad_samples.append(wsi_name)
    # print(bad_samples)
    # print(len(bad_samples))
    for wsi_name in bad_samples:
      self.wsi_names.remove(wsi_name)
    return slide_dict

  def sample_coord_list(self, pseudo_epoch_length):
    # # sample random coordinates
    # filenames, coords = self._sample_random_coords(pseudo_epoch_length)

    # sample nonempty coordinates
    filenames, coords = self._sample_nonempty_coords(pseudo_epoch_length)
    
    # bring everything in one dict
    sample_dict = {}
    for index, (filename, coord) in enumerate(zip(filenames, coords)):
      sample_dict[index] = {'filename': filename, 'coordinates': coord}

    return sample_dict

  def _sample_nonempty_coords(self, pseudo_epoch_length):
    filenames = []
    coords = []
    for i in range(pseudo_epoch_length):

      # Select random file, it's mask thumbnail, and get all non-empty coordinates
      filename = random.choice(self.wsi_names, size=1)[0]
      # mask_slide = self.slide_dict[filename]['mask']
      width, height = self.slide_dict[filename]['size']
      mask_thumbnail = self.mask_thumbnails[filename]
      indices = np.transpose(np.where(mask_thumbnail>0))
      # print(indices.size)
      while (indices.size==0):
        filename = random.choice(self.wsi_names, size=1)[0]
        # mask_slide = self.slide_dict[filename]['mask']
        width, height = self.slide_dict[filename]['size']
        mask_thumbnail = self.mask_thumbnails[filename]
        indices = np.transpose(np.where(mask_thumbnail>0))
      # print(indices.size)
      # plt.figure(figsize=(20,20))
      # plt.imshow(mask_thumbnail)
      # plt.show()

      # Pick random index and invert coordinates from (y,x) to (x,y)
      rand_index = random.randint(len(indices))
      coord = indices[rand_index][::-1]
      # print(mask_thumbnail[coord[1]][coord[0]])

      # Scale coord to wsi size and add a little randomness
      coord[0] = coord[0]*PATCH_WIDTH + random.randint(low=-PATCH_WIDTH//8,
                                                        high=PATCH_WIDTH//8)
      if coord[0]<0: coord[0]=0
      if coord[0]>width-PATCH_WIDTH: coord[0]=width-PATCH_WIDTH
      coord[1] = coord[1]*PATCH_HEIGHT + random.randint(low=-PATCH_HEIGHT//8,
                                                        high=PATCH_HEIGHT//8)
      if coord[1]<0: coord[1]=0
      if coord[1]>height-PATCH_HEIGHT: coord[1]=height-PATCH_HEIGHT

      # coord[0] = coord[0]*PATCH_WIDTH
      # coord[1] = coord[1]*PATCH_HEIGHT

      # print("coord: " + str(coord))
      # print("width: " + str(width) + ", height: " + str(height))

      filenames.append(filename)
      coords.append(coord)
    return filenames, coords

  # def _sample_random_coords(self, pseudo_epoch_length):
  #   filenames = list(random.choice(self.wsi_names, size=pseudo_epoch_length, replace=True))
  #   coords = []
  #   for filename in filenames: 
  #     width, height = self.slide_dict[filename]['size']
  #     xy = list(random.randint(low=(0, 0), 
  #                             high=(width-PATCH_WIDTH, height-PATCH_HEIGHT),
  #                             size=2))
  #     coords.append(xy)
  #   return filenames, coords

  def __len__(self):
    # return the number of total samples
    return self.pseudo_epoch_length

  def __getitem__(self, index):
    # grab the image from the current index
    coords = self.sample_dict[index]['coordinates'].copy()
    filename = self.sample_dict[index]['filename']

    # load patch and mask
    img, mask = self.load_image(filename, coords)

    img = self.transformations(img)
    # mask = self.transformations(mask)
    # mask = torch.as_tensor(mask) #.permute(1, 2, 0)

    # # Split mask into binary masks for each class
    # channels = np.tile(mask, (NUM_CLASSES, 1, 1))
    # for (i, channel) in enumerate(channels):
    #     channel = torch.as_tensor(channel).type(dtype=torch.float32)
    #     channels[i] = torch.tensor(1.0).where(channel==i+1, torch.tensor(0.0))
    # mask = channels

    return img, mask

  def load_image(self, filename, coords):
    """Loads an image and corresponding mask patch from a slide and returns it as a numpy array."""
    slide = self.slide_dict[filename]['slide']
    patch = slide.read_region(coords, size=(PATCH_WIDTH, PATCH_HEIGHT), level=0).convert('RGB')
    mask_slide = self.slide_dict[filename]['mask']
    mask_patch = mask_slide.read_region(coords, size=(PATCH_WIDTH, PATCH_HEIGHT), level=0).convert('RGB')
    # print(mask_patch.size)
    # print(np.mean(mask_patch))
    # print(np.asarray(mask_patch, dtype=np.uint8))
    return np.asarray(patch, dtype=np.uint8), np.asarray(mask_patch, dtype=np.uint8)[:,:,0]

### UNET Modules

In [4]:
class Block(Module):
	def __init__(self, inChannels, outChannels):
		super().__init__()
		# store the convolution and RELU layers
		self.conv1 = Conv2d(inChannels, outChannels, 3)
		self.relu = ReLU()
		self.conv2 = Conv2d(outChannels, outChannels, 3)
    
	def forward(self, x):
		# apply CONV => RELU => CONV block to the inputs and return it
		return self.conv2(self.relu(self.conv1(x)))

class Encoder(Module):
	def __init__(self, channels=(3, 16, 32, 64)):
		super().__init__()
		# store the encoder blocks and maxpooling layer
		self.encBlocks = ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])
		self.pool = MaxPool2d(2)
    
	def forward(self, x):
		# initialize an empty list to store the intermediate outputs
		blockOutputs = []
		# loop through the encoder blocks
		for block in self.encBlocks:
			# pass the inputs through the current encoder block, store
			# the outputs, and then apply maxpooling on the output
			x = block(x)
			blockOutputs.append(x)
			x = self.pool(x)
		# return the list containing the intermediate outputs
		return blockOutputs

class Decoder(Module):
	def __init__(self, channels=(64, 32, 16)):
		super().__init__()
		# initialize the number of channels, upsampler blocks, and
		# decoder blocks
		self.channels = channels
		self.upconvs = ModuleList(
			[ConvTranspose2d(channels[i], channels[i + 1], 2, 2)
			 	for i in range(len(channels) - 1)])
		self.dec_blocks = ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])

	def forward(self, x, encFeatures):
		# loop through the number of channels
		for i in range(len(self.channels) - 1):
			# pass the inputs through the upsampler blocks
			x = self.upconvs[i](x)
			# crop the current features from the encoder blocks,
			# concatenate them with the current upsampled features,
			# and pass the concatenated output through the current
			# decoder block
			encFeat = self.crop(encFeatures[i], x)
			x = torch.cat([x, encFeat], dim=1)
			x = self.dec_blocks[i](x)
		# return the final decoder output
		return x
    
	def crop(self, encFeatures, x):
		# grab the dimensions of the inputs, and crop the encoder
		# features to match the dimensions
		# print(x.shape)
		(_, _, H, W) = x.shape
		# H = x.shape[-2]
		# W = x.shape[-1]
		encFeatures = transforms.CenterCrop([H, W])(encFeatures)
		# return the cropped features
		return encFeatures
		
class UNet(Module):
	def __init__(self, encChannels=(3, 16, 32, 64),
		 decChannels=(64, 32, 16),
		 nbClasses=NUM_CLASSES, retainDim=True,
		 outSize=(PATCH_WIDTH, PATCH_HEIGHT)):
		super().__init__()
		# initialize the encoder and decoder
		self.encoder = Encoder(encChannels)
		self.decoder = Decoder(decChannels)
		# initialize the regression head and store the class variables
		self.head = Conv2d(decChannels[-1], nbClasses, 1)
		self.retainDim = retainDim
		self.outSize = outSize

	def forward(self, x):
		# grab the features from the encoder
		encFeatures = self.encoder(x)
		# pass the encoder features through decoder making sure that
		# their dimensions are suited for concatenation
		decFeatures = self.decoder(encFeatures[::-1][0],
			encFeatures[::-1][1:])
		# pass the decoder features through the regression head to
		# obtain the segmentation mask
		map = self.head(decFeatures)
		# check to see if we are retaining the original output
		# dimensions and if so, then resize the output to match them
		if self.retainDim:
			map = F.interpolate(map, self.outSize)
		# return the segmentation map
		return map

### Training

In [5]:
# Take only radboud slides
radboud = train.loc[train["data_provider"]=="radboud"]

# Take only wsi names
all__img_names = list(radboud.index)

# partition the data into training and testing splits using 95% of
# the data for training and the remaining 5% for testing
test_split_size = math.floor(TEST_SPLIT*len(all__img_names))
test_split = torch.utils.data.random_split(all__img_names,
									[test_split_size, len(all__img_names)-test_split_size], 
									generator=torch.Generator().manual_seed(42))

# unpack the data split
(all_train_img_names, test_img_names) = test_split
all_train_img_names = list(all_train_img_names)
test_img_names = list(test_img_names)

# partition the data into training and validation splits using 85% of
# the data for training and the remaining 15% for validation
split_size = math.floor(VAL_SPLIT*len(all_train_img_names))
split = torch.utils.data.random_split(all_train_img_names,
									[split_size, len(all_train_img_names)-split_size], 
									generator=torch.Generator().manual_seed(42))

# unpack the data split
(train_img_names, val_img_names) = split
train_img_names = list(train_img_names)
val_img_names = list(val_img_names)
# print(train_img_names[:10])
# print(val_img_names[:10])

# create the train and validation datasets
trainDS = SegmentationDataset(wsi_names=train_img_names, mask_thumbnails=thumbnails_dict, pseudo_epoch_length=NUM_PSEUDO_EPOCHS)
valDS = SegmentationDataset(wsi_names=val_img_names, mask_thumbnails=thumbnails_dict, pseudo_epoch_length=NUM_PSEUDO_EPOCHS)
print(f"[INFO] found {len(trainDS)} samples in the training set...")
print(f"[INFO] found {len(valDS)} samples in the validation set...")

# create the training and validation data loaders
trainLoader = DataLoader(trainDS, shuffle=True,
	batch_size=BATCH_SIZE, num_workers=4)
valLoader = DataLoader(valDS, shuffle=False,
	batch_size=BATCH_SIZE, num_workers=4)

Make Slide Dict: 100%|██████████| 736/736 [00:00<00:00, 1203.89it/s]


[INFO] found 1024 samples in the training set...
[INFO] found 1024 samples in the validation set...


In [6]:
# model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
#     in_channels=3, out_channels=1, init_features=32, pretrained=True)

In [7]:
# initialize our UNet model
unet = UNet().to(DEVICE)
# initialize loss function and optimizer
lossFunc = FocalLoss(0.25) #dice loss, focal loss
# lossFunc = BCEWithLogitsLoss()
opt = Adam(unet.parameters(), lr=INIT_LR)
# calculate steps per epoch for training and validation set
trainSteps = len(trainDS) // BATCH_SIZE
valSteps = len(valDS) // BATCH_SIZE
# initialize a dictionary to store training history
H = {"train_loss": [], "val_loss": []}

In [8]:
# Visualize slide patches and masks

# rows = 20

# all_figure, all_ax = plt.subplots(nrows=rows, ncols=2, figsize=(30, 30))

# for e in tqdm(range(rows)):
# 	# loop over the training set
# 	for (i, (x, y)) in enumerate(trainLoader):
# 		if i<1:
# 			all_ax[e,0].imshow(y[0], cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
# 			all_ax[e,1].imshow(torch.as_tensor(x[0]).permute(1, 2, 0), cmap=cmap, interpolation='nearest', vmin=0, vmax=5)

# all_figure.tight_layout()
# plt.show()

In [9]:
# def compressMask(channels):
#     mask = np.zeros((PATCH_WIDTH, PATCH_HEIGHT))
#     for (i, channel) in enumerate(channels):
#         mask += channel*(i+1)
#     return mask

# def compressPredMask(channels):
#     mask = np.zeros((PATCH_WIDTH, PATCH_HEIGHT))
#     for (i, channel) in enumerate(channels):
#         mask += (0-np.rint(channel.numpy()/256))*(i+1)
#     return mask

In [10]:
# loop over epochs
print("[INFO] training the network...")
startTime = time.time()
for e in tqdm(range(NUM_EPOCHS)):
	# set the model in training mode
	unet.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0

	# loop over the training set
	for (i, (x, y)) in enumerate(trainLoader):
		# if i<1:
		# 	all_figure, all_ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 30))
		# 	realMask = y[0].numpy()
		# 	all_ax[1].imshow(realMask, cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
		# 	all_ax[0].imshow(torch.as_tensor(x[0]).permute(1, 2, 0), cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
		# send the input to the device
		(x, y) = (x.to(DEVICE), y.to(DEVICE))
		# perform a forward pass and calculate the training loss
		pred = unet(x).squeeze() #.type(dtype=torch.uint8)
		y = y.squeeze()
		y = y.type(dtype=torch.long)
		# print(f'preds {pred.shape}')
		# print(f'y shape: {y.shape}')
		loss = lossFunc(pred, y)
		# print(f'loss: {loss}')
		# print(f'loss shape: {loss.shape}')
		# first, zero out any previously accumulated gradients, then
		# perform backpropagation, and then update model parameters
		opt.zero_grad()
		loss.backward()
		opt.step()
		# if i<1:
		# 	out = torch.argmax(pred[0], dim=0)
		# 	all_ax[2].imshow(out.cpu().detach().numpy(), cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
		# 	all_figure.tight_layout()
		# 	plt.show()
		# add the loss to the total training loss so far
		totalTrainLoss += loss
    
	# switch off autograd
	with torch.no_grad():
		# set the model in evaluation mode
		unet.eval()
		# loop over the validation set
		for (x, y) in valLoader:
			# send the input to the device
			(x, y) = (x.to(DEVICE), y.to(DEVICE))
			# make the predictions and calculate the validation loss
			pred = unet(x).squeeze() #.type(dtype=torch.uint8)
			y = y.squeeze()
			y = y.type(dtype=torch.long)
			totalValLoss += lossFunc(pred, y)

	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, NUM_EPOCHS))
	print("Train loss: {:.6f}, Val loss: {:.4f}".format(
		avgTrainLoss, avgValLoss))

# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))

[INFO] training the network...


  2%|▎         | 1/40 [00:10<07:08, 10.98s/it]

[INFO] EPOCH: 1/40
Train loss: 0.178198, Val loss: 0.1401


  5%|▌         | 2/40 [00:21<06:50, 10.80s/it]

[INFO] EPOCH: 2/40
Train loss: 0.134229, Val loss: 0.1347


...

[INFO] EPOCH: 37/40
Train loss: 0.084289, Val loss: 0.0838


 95%|█████████▌| 38/40 [06:46<00:21, 10.66s/it]

[INFO] EPOCH: 38/40
Train loss: 0.085717, Val loss: 0.0843


 98%|█████████▊| 39/40 [06:57<00:10, 10.65s/it]

[INFO] EPOCH: 39/40
Train loss: 0.085754, Val loss: 0.0858


In [ ]:
# plot the training loss
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")

In [ ]:
plt.savefig(PLOT_PATH)
# serialize the model to disk
# torch.save(unet, MODEL_PATH)

## Testing

In [ ]:
figure, ax = plt.subplots(nrows=1, ncols=4, figsize=(10, 10))

test_img_name = test_img_names[1]
# print(test_img_name)

test_slide_path = os.path.join(data_dir, f'{test_img_name}.tiff')
test_slide = openslide.OpenSlide(test_slide_path)
test_slide_size = test_slide.dimensions
# print(test_slide_size)
thumbnail = test_slide.get_thumbnail((test_slide_size[0]/PATCH_WIDTH, test_slide_size[1]/PATCH_HEIGHT)).convert('RGB')
ax[0].imshow(thumbnail)

test_coords = [57*PATCH_WIDTH,28*PATCH_HEIGHT]
test_patch = test_slide.read_region(test_coords, size=(PATCH_WIDTH, PATCH_HEIGHT), level=0).convert('RGB')
test_patch = np.asarray(test_patch, dtype=np.uint8)
ax[1].imshow(test_patch)

test_mask_path = os.path.join(mask_dir, f'{test_img_name}_mask.tiff')
test_mask = openslide.OpenSlide(test_mask_path)
test_mask_patch = test_mask.read_region(test_coords, size=(PATCH_WIDTH, PATCH_HEIGHT), level=0).convert('RGB')
test_mask_patch = np.asarray(test_mask_patch, dtype=np.uint8)[:,:,0]
ax[2].imshow(test_mask_patch, cmap=cmap, interpolation='nearest', vmin=0, vmax=5)

unet.eval()
with torch.no_grad():
    # transformations = transforms.Compose([transforms.ToTensor()])
    # # print(test_patch.shape)
    test_patch = np.transpose(test_patch, (2,0,1))
    # test_patch = np.expand_dims(test_patch, 0)
    # test_patch = transformations(test_patch)
    # print(test_patch.shape)
    # Split mask into binary masks for each class
    # channels = np.tile(test_patch, (NUM_CLASSES, 1, 1))
    # for (i, channel) in enumerate(channels):
    #     channel = torch.as_tensor(channel).type(dtype=torch.float32)
    #     channels[i] = torch.tensor(1.0).where(channel==i+1, torch.tensor(0.0))
    # test_patch = torch.from_numpy(channels).to(DEVICE)
    test_patch = torch.from_numpy(test_patch).to(DEVICE).unsqueeze(0)
    print(test_patch.shape)
    test_patch = test_patch.type(dtype=torch.float32)
    predMask = unet(test_patch).squeeze()
    predMask = torch.argmax(predMask, dim=0)
    predMask = predMask.cpu().detach().numpy()
    ax[3].imshow(predMask, cmap=cmap, interpolation='nearest', vmin=0, vmax=5)

figure.tight_layout()
plt.show()

In [ ]:
# create the test dataset
testDS = SegmentationDataset(wsi_names=test_img_names, mask_thumbnails=thumbnails_dict, pseudo_epoch_length=NUM_PSEUDO_EPOCHS)
print(f"[INFO] found {len(testDS)} samples in the test set...")

# create the test data loader
testLoader = DataLoader(testDS, shuffle=True,
	                    batch_size=BATCH_SIZE, num_workers=4)

# calculate steps per epoch for test set
testSteps = len(testDS) // BATCH_SIZE

# loop over epochs
print("[INFO] testing the network...")
startTime = time.time()
# initialize the total test loss
totalTestLoss = 0
# switch off autograd
with torch.no_grad():
    # set the model in evaluation mode
    unet.eval()
    for (i, (x, y)) in tqdm(enumerate(testLoader)):
        # send the input to the device
        (x, y) = (x.to(DEVICE), y.to(DEVICE))
        # make the predictions and calculate the test loss
        pred = unet(x).squeeze()
        y = y.squeeze().type(dtype=torch.long)
        totalTestLoss += lossFunc(pred, y)
        for i in range(BATCH_SIZE):
            all_figure, all_ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 20))
            all_ax[0].imshow(torch.as_tensor(x[i].cpu().detach().numpy()).permute(1, 2, 0), cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
            realMask = y[i].cpu().detach().numpy()
            all_ax[1].imshow(realMask, cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
            predMask = torch.argmax(pred[i], dim=0)
            predMask = predMask.cpu().detach().numpy()
            all_ax[2].imshow(predMask, cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
            all_figure.tight_layout()
            plt.show()

# calculate the average test loss
avgTestLoss = (totalTestLoss / testSteps).cpu().detach().numpy()
print("Test loss: {avgTestLoss}")

# display the total time needed to perform the testing
endTime = time.time()
print("[INFO] total time taken to test the model: {:.2f}s".format(endTime - startTime))